In [33]:
import pandas as pd
import numpy as np

In [34]:
files_to_merge = [
    "EURUSD_1m_raw_2020-11-22_2020-12-31.csv",
    "EURUSD_1m_raw_2021-01-01_2021-12-31.csv",
    "EURUSD_1m_raw_2022-01-01_2022-12-31.csv",
    "EURUSD_1m_raw_2023-01-01_2023-12-31.csv",
    "EURUSD_1m_raw_2024-01-01_2024-12-31.csv",
    "EURUSD_1m_raw_2025-01-01_2025-11-21.csv",
]



In [35]:
for file in files_to_merge:
    df_temp=pd.read_csv(file)
    print(f"file {file} loaded, shape: {df_temp.shape}")

file EURUSD_1m_raw_2020-11-22_2020-12-31.csv loaded, shape: (40622, 6)
file EURUSD_1m_raw_2021-01-01_2021-12-31.csv loaded, shape: (370681, 6)
file EURUSD_1m_raw_2022-01-01_2022-12-31.csv loaded, shape: (372879, 6)
file EURUSD_1m_raw_2023-01-01_2023-12-31.csv loaded, shape: (371105, 6)
file EURUSD_1m_raw_2024-01-01_2024-12-31.csv loaded, shape: (372798, 6)
file EURUSD_1m_raw_2025-01-01_2025-11-21.csv loaded, shape: (333186, 6)


In [37]:
dfs=[pd.read_csv(f) for f in files_to_merge]
df=pd.concat(dfs, ignore_index=True)
print(f"All files merged, final shape: {df.shape}")

All files merged, final shape: (1861271, 6)


In [38]:
print(df['timestamp'].iloc[0])
print(df['timestamp'].iloc[-1])

2020-11-22 22:00:00+00:00
2025-11-21 21:59:00+00:00


last value here is 21 Nov 2025 because 22 Nov was a saturday hence 0 raw

In [39]:
#renaming the columns as required
df = df.rename(columns={
    "UTC": "timestamp",
    "Open": "open",
    "High": "high",
    "Low": "low",
    "Close": "close",
    "Volume": "volume",
})
df.head(2)

,timestamp,open,high,low,close,volume
0,2020-11-22 22:00:00+00:00,1.18538,1.18547,1.18537,1.18541,10.50
1,2020-11-22 22:01:00+00:00,1.18540,1.18542,1.18537,1.18541,26.25


In [40]:
#parsing timestamp to UTC datetime
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, dayfirst=True)
df.head(2)

C:\Users\bessa\AppData\Local\Temp\ipykernel_9960\3512892983.py:2: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, dayfirst=True)


,timestamp,open,high,low,close,volume
0,2020-11-22 22:00:00+00:00,1.18538,1.18547,1.18537,1.18541,10.50
1,2020-11-22 22:01:00+00:00,1.18540,1.18542,1.18537,1.18541,26.25


In [41]:
# 2) Sort and drop duplicate timestamps
df = df.sort_values("timestamp").drop_duplicates(subset=["timestamp"]).reset_index(drop=True)
print(f"min {df['timestamp'].min()}, max {df['timestamp'].max()}")


min 2020-11-22 22:00:00+00:00, max 2025-11-21 21:59:00+00:00


In [42]:
# Build expected 1‑minute continuous interval
full_index = pd.date_range(
    start=df["timestamp"].min(),
    end=df["timestamp"].max(),
    freq="min",   # 1 minute
    tz="UTC",
)
# Reindex onto this grid
df = df.set_index("timestamp").reindex(full_index)

print("full_index length:", len(full_index))
print("df index length  :", len(df))
print(df.index.min(), df.index.max())


full_index length: 2628000
df index length  : 2628000
2020-11-22 22:00:00+00:00 2025-11-21 21:59:00+00:00


In [43]:
df["missing_bar"] = df["open"].isna()
df["missing_bar"].value_counts()

missing_bar
False    1861271
True      766729
Name: count, dtype: int64

In [44]:
# Gap summary values 
earliest = df.index.min()   # timestamp is the index
latest = df.index.max()
expected = len(df)          # 1861271 continuous minutes
actual = (df["missing_bar"] == False).sum()  # 2628000 real bars
filled = (df["missing_bar"] == True).sum()   # 766729 synthetic bars

print("Earliest timestamp:", earliest)
print("Latest timestamp :", latest)
print("Expected 1m bars :", expected)
print("Actual raw bars  :", actual)
print("Missing minutes filled (missing_bar=True):", filled)

Earliest timestamp: 2020-11-22 22:00:00+00:00
Latest timestamp : 2025-11-21 21:59:00+00:00
Expected 1m bars : 2628000
Actual raw bars  : 1861271
Missing minutes filled (missing_bar=True): 766729


In [45]:
# Save original missing timestamps to a text file (before fill)
missing_ts = df.index[df["missing_bar"] == True]
missing_ts.to_series().dt.strftime("%Y-%m-%dT%H:%M:%SZ").to_csv(
    "EURUSD_1m_UTC_2020-11-22_2025-11-21_missing_timestamps.txt",
    index=False,
    header=False,
)

In [46]:
# Fill missing minutes using prior bar's OHLCV (ffill)
df[["open", "high", "low", "close", "volume"]] = (
    df[["open", "high", "low", "close", "volume"]].ffill()
)

In [47]:
df = df.reset_index().rename(columns={"index": "timestamp"})
df = df[df["timestamp"].dt.dayofweek < 5]

In [48]:
#  Format to ISO8601 UTC string
df["timestamp"] = df["timestamp"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

In [49]:
# Enforce final order
cols = ["timestamp", "open", "high", "low", "close", "volume", "missing_bar"]
df = df[cols]

This dataset uses Dukascopy EURUSD 1‑minute data for a rolling 5‑year window,
Raw downloads request to 2025-11-22 23:59
but Dukascopy provides up to Friday 2025-11-21 21:59 market close (Saturday exclude), aligned to a continuous 1‑minute UTC grid between the earliest and latest timestamp in the raw download.
Timestamps were parsed as UTC, sorted, deduplicated, and aligned to this continuous 1‑minute grid. Missing minutes were filled using the prior bar’s OHLCV values (ffill), and a missing_bar boolean column flags all synthetic bars created by this procedure, following the “fill missing minutes using prior close + missing_bar” option in the brief. Weekend rows (Saturday and Sunday) were removed from the final delivered file, while preserving the gap statistics on trading days

In [50]:
# Save final 5‑year EURUSD csv
output_path = "EURUSD_1m_UTC_2020-11-22_2025-11-21.csv"
df.to_csv(output_path, index=False)

In [ ]:
#metadata json 
import json

df_final = pd.read_csv("EURUSD_1m_UTC_2020-11-22_2025-11-21.csv")

final_total = len(df_final)  # 1,879,080
final_filled = (df_final["missing_bar"] == True).sum()  # 57,688

# Update metadata
meta = {
    "pair": "EURUSD",
    "from": "2020-11-22T22:00:00Z",
    "to": "2025-11-21T21:59:00Z",
    "total_rows": int(final_total),
    "missing_bars": int(final_filled),
    "source": "Dukascopy",
    "note": (
        "Rolling 5-year weekday-only data (Sat/Sun excluded); "
        "missing weekday minutes forward-filled and flagged via missing_bar."
    ),
}

with open("EURUSD_1m_UTC_2020-11-22_2025-11-21_meta.json", "w") as f:
    json.dump(meta, f, indent=2)


In [ ]:
# Update gap summary
summary = f"""EURUSD 1-minute Dukascopy (UTC)

Coverage:
- From: 2020-11-22 22:00:00 UTC
- To  : 2025-11-21 21:59:00 UTC

Counts (Weekdays Only):
- Total rows in file: {final_total}
- Real bars: {final_total - final_filled}
- Filled bars: {final_filled}

Notes:
- Weekends excluded.
- Missing weekday minutes forward-filled.

"""

with open("EURUSD_1m_UTC_2020-11-22_2025-11-22_gap_summary.txt", "w") as f:
    f.write(summary)

print("✓ Metadata updated to match file.")

✓ Metadata updated to match file.


In [53]:
# Quick validation check after weekend filter

print("=== EURUSD 5-Year Dataset Validation ===\n")

# 1. File check
import os
files_expected = [
    "EURUSD_1m_UTC_2020-11-22_2025-11-21.csv",
    "EURUSD_1m_UTC_2020-11-22_2025-11-21_gap_summary.txt",
    "EURUSD_1m_UTC_2020-11-22_2025-11-21_meta.json",
    "EURUSD_1m_UTC_2020-11-22_2025-11-21_missing_timestamps.txt",
]
print("File outputs:")
for f in files_expected:
    exists = "✓" if os.path.exists(f) else "✗ MISSING"
    print(f"  {f}: {exists}")

# 2. CSV integrity
df_final = pd.read_csv("EURUSD_1m_UTC_2020-11-22_2025-11-21.csv")
print(f"\nCSV shape: {df_final.shape}")
print(f"Columns: {df_final.columns.tolist()}")
print(f"First row timestamp: {df_final['timestamp'].iloc[0]}")
print(f"Last row timestamp: {df_final['timestamp'].iloc[-1]}")

# 3. Data quality
weekday_count = df_final[df_final['missing_bar'] == False].shape[0]
synthetic_count = df_final[df_final['missing_bar'] == True].shape[0]
print(f"\nData composition:")
print(f"  Real bars (missing_bar=False): {weekday_count}")
print(f"  Synthetic bars (missing_bar=True): {synthetic_count}")
print(f"  Total rows: {len(df_final)}")
print(f"  NaN values in OHLCV: {df_final[['open','high','low','close','volume']].isna().sum().sum()}")

# 4. Metadata check
import json
with open("EURUSD_1m_UTC_2020-11-22_2025-11-21_meta.json") as f:
    meta = json.load(f)
print(f"\nMetadata:")
print(f"  Pair: {meta['pair']}")
print(f"  From: {meta['from']}")
print(f"  To: {meta['to']}")
print(f"  Total rows: {meta['total_rows']}")
print(f"  Missing bars: {meta['missing_bars']}")
print(f"  Source: {meta['source']}")

print("\n✓ EURUSD 5-year build complete and validated.")

=== EURUSD 5-Year Dataset Validation ===

File outputs:
  EURUSD_1m_UTC_2020-11-22_2025-11-21.csv: ✓
  EURUSD_1m_UTC_2020-11-22_2025-11-21_gap_summary.txt: ✓
  EURUSD_1m_UTC_2020-11-22_2025-11-21_meta.json: ✓
  EURUSD_1m_UTC_2020-11-22_2025-11-21_missing_timestamps.txt: ✓

CSV shape: (1879080, 7)
Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'missing_bar']
First row timestamp: 2020-11-23T00:00:00Z
Last row timestamp: 2025-11-21T21:59:00Z

Data composition:
  Real bars (missing_bar=False): 1821392
  Synthetic bars (missing_bar=True): 57688
  Total rows: 1879080
  NaN values in OHLCV: 0

Metadata:
  Pair: EURUSD
  From: 2020-11-22T22:00:00Z
  To: 2025-11-21T21:59:00Z
  Total rows: 1879080
  Missing bars: 57688
  Source: Dukascopy

✓ EURUSD 5-year build complete and validated.
